In [87]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/wikiart-damage-square/test.h5
/kaggle/input/wikiart-damage-square/train.h5
/kaggle/input/wikiart-damage-square/valid.h5


In [88]:
# !pip -q install lightning
# !pip -q install comet-ml

In [89]:
import h5py
import torch
import numpy as np
from PIL import Image
import math
import os

import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import torchvision.utils as vutils
from torchvision.transforms import v2

import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping

In [90]:
from lightning.pytorch.loggers import CometLogger
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("COMET_API_KEY")

comet_logger = CometLogger(
    api_key=secret_value_0,
    project_name='UNet_Inpainting',
    workspace='wikiart-inpainting',
)

INFO: CometLogger will be initialized in online mode


In [91]:
class WikiArtDataset(Dataset):
    def __init__(self, h5_path: str, transform=None, mask_transform=None):
        """
        A dataset for WikiArt data integrated with HDF5 image and mask reading.

        Parameters:
            h5_path (str): Path to the HDF5 file containing images and masks.
            transform (callable, optional): A function to transform the images.
            mask_transform (callable, optional): A function to transform the masks.
        """
        self.h5_path = h5_path
        self.transform = transform
        self.mask_transform = mask_transform

        with h5py.File(self.h5_path, 'r') as h5f:
            self.length = len(h5f['images'])

    def __len__(self):
        """
        Returns the total number of elements in the dataset.

        Returns:
            int: The number of images in the dataset.
        """
        return self.length

    def _open_hdf5(self):
        """
        Opens the HDF5 file if it hasn't been opened yet.
        """
        if not hasattr(self, '_hf') or self._hf is None:
            self._hf = h5py.File(self.h5_path, 'r')

    def __getitem__(self, idx):
        """
        Retrieves an image and its corresponding mask based on the given index.

        Parameters:
            idx (int): The index of the element to retrieve.

        Returns:
            tuple: A tuple containing the image (torch.Tensor) and the mask (torch.Tensor).
        """
        self._open_hdf5()

        image = self._hf['images'][idx]
        mask = self._hf['masks'][idx]

        image = torch.from_numpy(image).float()
        mask = torch.from_numpy(mask).float()

        if self.transform:
            image = self.transform(image)

        if self.mask_transform:
            mask = self.mask_transform(mask)

        return image, mask

In [92]:
class UNetInpainting(nn.Module):
    def __init__(self, in_channels=3, out_channels=3, use_dropout=False):
        super().__init__()
        self.use_dropout = use_dropout

        self.encoder1 = self.conv_block(in_channels, 16)
        self.encoder2 = self.conv_block(16, 32, pool=True)
        self.encoder3 = self.conv_block(32, 64, pool=True)
        self.encoder4 = self.conv_block(64, 128, pool=True)

        self.bottleneck = self.conv_block(128, 256, pool=True)

        self.decoder4 = self.upconv_block(256, 128)
        self.decoder3 = self.upconv_block(128, 64)
        self.decoder2 = self.upconv_block(64, 32)
        self.decoder1 = self.upconv_block(32, 16)

        self.final_conv = nn.Conv2d(16, out_channels, kernel_size=1)

    def conv_block(self, in_channels, out_channels, pool=False):
        layers = []
        if pool:
            layers.append(nn.MaxPool2d(kernel_size=2))
        layers.extend([
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(out_channels),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(out_channels)
        ])
        if self.use_dropout:
            layers.append(nn.Dropout(0.5))
        return nn.Sequential(*layers)

    def upconv_block(self, in_channels, out_channels):
        return nn.Sequential(
            nn.ConvTranspose2d(in_channels, out_channels, kernel_size=2, stride=2),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(out_channels)
        )

    def forward(self, x, mask):
        masked_image = x * (1 - mask)

        e1 = self.encoder1(masked_image)  # No downsampling here
        e2 = self.encoder2(e1)
        e3 = self.encoder3(e2)
        e4 = self.encoder4(e3)

        b = self.bottleneck(e4)

        d4 = self.decoder4(b) + e4
        d3 = self.decoder3(d4) + e3
        d2 = self.decoder2(d3) + e2
        d1 = self.decoder1(d2) + e1

        output = self.final_conv(d1)

        return output

In [93]:
def calculate_psnr(output, target):
    mse = F.mse_loss(output, target, reduction='mean')
    if mse == 0:
        return 100
    psnr = 20 * math.log10(1.0 / math.sqrt(mse.item()))
    return psnr

In [94]:
def log_images_to_comet(logger, images, mask, output, epoch, step, name='sample'):
    grid = vutils.make_grid(
        [images[0].cpu(), output[0].cpu(), (images[0] * (1 - mask[0])).cpu()],
        nrow=3,
        normalize=True
    )

    grid_np = grid.permute(1, 2, 0).detach().numpy()  # CxHxW -> HxWxC

    grid_image = Image.fromarray((grid_np * 255).clip(0, 255).astype('uint8'))

    logger.experiment.log_image(
        grid_image, name=f'{name}_epoch_{epoch}_step_{step}.png'
    )

In [95]:
class UNetInpaintingLightning(pl.LightningModule):
    def __init__(self, model):
        super().__init__()
        self.model = model
        self.criterion = nn.MSELoss()
        self.best_val_loss = float('inf')
        self.automatic_optimization = False

    def forward(self, x, mask):
        return self.model(x, mask)

    def training_step(self, batch, batch_idx):
        images, masks = batch
        mask = masks.unsqueeze(1)
        
        optimizer = self.optimizers()
        optimizer.zero_grad(set_to_none=True)
        
        with torch.amp.autocast('cuda'):
            output = self(images, mask)
            loss = self.criterion(output, images)
        
        self.manual_backward(loss)
        optimizer.step()
        
        self.log('train_loss', loss, prog_bar=True, on_epoch=True, on_step=True)

        if batch_idx % 200 == 0:
            log_images_to_comet(
                self.logger,
                images,
                mask,
                output,
                epoch=self.current_epoch,
                step=self.global_step,
                name='train_sample'
            )

        return loss

    def validation_step(self, batch, batch_idx):
        images, masks = batch
        mask = masks.unsqueeze(1)
        output = self(images, mask)
        loss = self.criterion(output, images)
    
        psnr = calculate_psnr(output, images)
        self.log('val_loss', loss, prog_bar=True, on_epoch=True, on_step=True)
        self.log('val_psnr', psnr, prog_bar=True, on_epoch=True, on_step=True)
    
        if batch_idx == 0:
            log_images_to_comet(
                self.logger,
                images,
                mask,
                output,
                epoch=self.current_epoch,
                step=self.global_step,
                name='val_sample'
            )
    
        return loss

    def configure_optimizers(self):
        optimizer = optim.Adam(self.model.parameters(), lr=1e-4)
        return optimizer

    def on_validation_epoch_end(self):
        val_loss = self.trainer.callback_metrics.get('val_loss')
        if val_loss is not None and val_loss < self.best_val_loss:
            self.best_val_loss = val_loss

            weights_path = f'best_weights_epoch_{self.current_epoch}_val_loss_{val_loss:.4f}.pth'
            torch.save(self.model.state_dict(), weights_path)

            self.logger.experiment.log_model(
                name='best_weights',
                file_or_folder=weights_path,
                overwrite=True
            )
            print(f'Weights saved and logged to CometML: {weights_path}')

In [96]:
transforms = v2.Compose([
    v2.RandomHorizontalFlip(),
    # v2.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    v2.ToDtype(torch.float32, scale=True),
    v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [97]:
dataset_path = '/kaggle/input/wikiart-damage-square/'

train_dataset = WikiArtDataset(h5_path=f'{dataset_path}train.h5')
valid_dataset = WikiArtDataset(h5_path=f'{dataset_path}valid.h5')
test_dataset = WikiArtDataset(h5_path=f'{dataset_path}test.h5')

In [98]:
batch_size = 32

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=batch_size, num_workers=2)

In [99]:
model = UNetInpainting()
lightning_model = UNetInpaintingLightning(model)

checkpoint_callback = ModelCheckpoint(
    monitor='val_loss',
    mode='min',
    dirpath='./checkpoints',
    filename='best_model-{epoch:02d}-{val_loss:.2f}',
    save_top_k=1,
    verbose=True
)

early_stopping_callback = EarlyStopping(
    monitor='val_loss',
    min_delta=0.00,
    patience=6,
    verbose=True,
    mode='min',
    check_on_train_epoch_end=False
)

trainer = pl.Trainer(
    logger=comet_logger,
    max_epochs=100,
    devices=1 if torch.cuda.is_available() else 0,
    callbacks=[checkpoint_callback, early_stopping_callback]
)

trainer.fit(lightning_model, train_loader, valid_loader)

INFO: GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs
COMET WARNING: To get all data logged automatically, import comet_ml before the following modules: torch.
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Couldn't find a Git repository in '/kaggle/working' nor in any parent directory. Set `COMET_GIT_DIRECTORY` if your Git Repository is elsewhere.
COMET INFO: Experiment is live on comet.com https://www.comet.com/wikiart-inpainting/unet-inpainting/643552ca1b1f48d899d25c5293397ec9

/opt/conda/lib/python3.10/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:654: Checkpoint directory /kaggle/working/checkpoints exists and is not empty.


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Weights saved and logged to CometML: best_weights_epoch_0_val_loss_14839.1309.pth


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

INFO: Epoch 98, global step 21186: 'val_loss' reached 37.23260 (best 37.23260), saving model to '/kaggle/working/checkpoints/best_model-epoch=98-val_loss=37.23.ckpt' as top 1


Weights saved and logged to CometML: best_weights_epoch_98_val_loss_37.2326.pth


Validation: |          | 0/? [00:00<?, ?it/s]

INFO: Epoch 99, global step 21400: 'val_loss' was not in top 1
INFO: `Trainer.fit` stopped: `max_epochs=100` reached.
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : divine_clownfish_730
COMET INFO:     url                   : https://www.comet.com/wikiart-inpainting/unet-inpainting/643552ca1b1f48d899d25c5293397ec9
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     train_loss_epoch [100] : (93.01545715332031, 16757.5546875)
COMET INFO:     train_loss_step [428]  : (28.7938232421875, 22489.66796875)
COMET INFO:     val_loss_epoch [100]   : (37.23259735107422, 16799.94140625)
COMET INFO:     val_loss_step [6600]   : (23.839889526367188, 21259.12109375)
COMET INFO:     val_psnr_epoch [100]   :

In [100]:
comet_logger.experiment.end()

COMET WARNING: To get all data logged automatically, import comet_ml before the following modules: torch.
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/wikiart-inpainting/unet-inpainting/643552ca1b1f48d899d25c5293397ec9

COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml ExistingExperiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : divine_clownfish_730
COMET INFO:     url                   : https://www.comet.com/wikiart-inpainting/unet-inpainting/643552ca1b1f48d899d25c5293397ec9
COMET INFO:   Others:
COMET INFO:     Created from : pytorch-lightning
COMET INFO: 
C